# CO2 time series analysis

## 1.1 Problem description

Since 1958 atmospheric carbon dioxide measurements have been recorded at the Mauna Loa Observatory in Hawaii. CO2 levels have been increasing steadily since the start of the industrial revolution in the 18th century.

Older data are from ice core measurements, not atmospheric measurements. The data from Mauna Loa provide very direct data on atmospheric CO2, which forms an important part of global climate change modeling.

Your task is to create a statistical model that explains this data set well and to use it to forecast what measurements will look like between now and the start of 2060 — 40 years from now. Your model should reflect the uncertainty you have in your predictions, showing **confidence intervals widening** as we go further away from the present time. You will also use the model to predict when we are likely to reach high-risk levels of CO2 with a greater probability of serious climate change.

You are required to model three components — an overall trend, seasonal variations, and noise. You will need to identify the parameters of your model, put priors over them, and calculate posterior distributions so that you can predict what atmospheric CO2 levels will look like up to the start of 2060.

## 1.2 Import libraries

In [1]:
import pandas as pd
import numpy as np
import pystan
import math
import random
import scipy.stats as sts

from scipy import optimize
from sklearn.metrics import r2_score
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline

# 2.0 Data preprocessing

## 2.1 Import data

[Atmospheric CO2 concentrations (ppm) derived from in situ air measurements at Mauna Loa, Observatory, Hawaii](https://scrippsco2.ucsd.edu/assets/data/atmospheric/stations/in_situ_co2/weekly/weekly_in_situ_co2_mlo.csv): Latitude 19.5N Longitude 155.6W Elevation 3397m

The data file below contains 2 columns indicaing the date and CO2 concentrations in micro-mol CO2 per mole (ppm), reported on the 2008A SIO manometric mole fraction scale.  These weekly values have been adjusted to 12:00 hours at middle day of each weekly period as indicated by the date in the first column.

In [9]:
df = pd.read_csv('weekly_in_situ_co2_mlo.csv',names=['date','CO2 ppm'], skiprows=44)
print('Dataframe has %d data points of %d features'
      %(df.shape[0],df.shape[1]))
df.head(50)

Dataframe has 3184 data points of 2 features


,date,CO2 ppm
0,1958-03-29,316.19
1,1958-04-05,317.31
2,1958-04-12,317.69
3,1958-04-19,317.58
4,1958-04-26,316.48
5,1958-05-03,316.95
6,1958-05-17,317.56
7,1958-05-24,317.99
8,1958-07-05,315.85
9,1958-07-12,315.85


## 2.2 Convert date column into 'datetime' format

This allows the plot to be formatted more nicely

Technically the series is in `Timestamp` format, which is Pandas' way of using `datetime` objects

In [10]:
df['date'] = pd.to_datetime(df['date'],format='%Y/%m/%d')
df.head()

,date,CO2 ppm
0,1958-03-29,316.19
1,1958-04-05,317.31
2,1958-04-12,317.69
3,1958-04-19,317.58
4,1958-04-26,316.48


## 2.3 Explore data

## 2.4 Reformat date

Reformat the date column into days, with the first date (1958 Mar. 29th) being day 0. 

# 3.0 Creating and Assessing Analytical Models

## 3.1 Proposed model (linear base trend)

To get you started here is an example of what such a model might look like. This model is not particularly good and you should come up with something more accurate.

* **time**: $t$ expressed in days
* **Long-term trend**: Linear: $c_0 + c_1 t$
* **Seasonal variation**: Sinusoidal: $c_2 \cos(2 \pi t / 365.25 + c_3)$
* **Noise**: Gaussian: $N(0,c_4)$
* The $c_i$ variables are all unobserved parameters of the model

Combining these three components gives the following likelihood function

$$
p(x_t | \theta) = N(c_0 + c_1 t + c_2 \cos(2 \pi t / 365.25 + c_3), c_4^2)
$$

where $\theta$ represents the set of all unobserved parameters. Since there are 3156 data, the full likelihood comprises a product overall 3156 values, $x_t$. To complete the model we would still need to define
priors over all 5 model parameters.

**Use np.polyfit(x,y,deg=1)**

## 3.2 Evaluate proposed model

Because the proposed model is so simple, I can conduct a simple observation for a "sanity check" of the model. I can create a **linear trend** (as proposed in the model) and see if the rest of the data can be **fully accounted for** by sinusoidal and noise parameters

## 3.3 Propose a second model (quadratic base trend)

As can be graphically observed, there is a seprate trend that cannot be accounted by any of the factors (linear, sinusoidal, and noise). To account for this, I constructed a new model whose base is **quadratic** than linear.

* **time**: $t$ expressed in days
* **Long-term trend**: Quadratic: $c_0 + c_1 t + c_2 t^2$
* **Seasonal variation**: Sinusoidal: $c_3 \cos(2 \pi t / 365.25 + c_4)$
* **Noise**: Gaussian: $N(0,c_5)$
* The $c_i$ variables are all unobserved parameters of the model

Combining these three components gives the following likelihood function

$$
p(x_t | \theta) = N(c_0 + c_1 t + c_2 t^2 + c_3 \cos(2 \pi t / 365.25 + c_4), c_5^2)
$$

In [40]:
def test_func(t,c0,c1,c2,c3,c4):
    return c0+c1*t+c2*t**2+c3*np.cos(2*math.pi*f_year*t+c4)


## 3.4 Evaluate second proposed model

The quadratic base trend looks more promising, but there still seems to be an underlying trend that cannot be **explained away** by sinusoidal and noise parameters. But the sinusoidal seasonal variations are too great for me to identify what that **unaccounted for trend** may be. I tried to evaluate by removing all the **accounted for** variations away from the data.

The model is still simple enough for me to attempt this through the `scipy.optimize` module

## 3.5 Propose a third model (exponential base trend)

The quadratic + sinusoidal model work beautifully, demonstrating an r-squared value of 0.999. The caveat is that my model may be **overfitting** to my data - the quadratic and sinusoidal model has a total of **5 parameters**. Is it possible to reduce a parameter but still achieve similar, if not better, results?

For this, I propose an **exponential function**. A key property of the exponential function is that vertical and horizontal shifts are interchangeable - hence can be expressed by a single parameter. The resulting components would look like the following:

* **time**: $t$ expressed in days
* **Long-term trend**: Exponential: $c_0 + e^{c_1t}$
* **Seasonal variation**: Sinusoidal: $c_2 \cos(2 \pi t / 365.25 + c_3)$
* **Noise**: Gaussian: $N(0,c_4)$
* The $c_i$ variables are all unobserved parameters of the model

Combining these three components gives the following likelihood function

$$
p(x_t | \theta) = N(c_0 + c_1e^t + c_2 \cos(2 \pi t / 365.25 + c_3), c_4^2)
$$

In [ ]:
def test_func(t,c0,c1,c2,c3):
    return c0+np.exp(c1*t)+c2*np.cos(2*math.pi*f_year*t+c3)

## 3.7 Propose a fourth model (increasing sinusoidal seasonal trend)

In [ ]:
def test_func(t,c0,c1,c2,c3,c4,c5):
    return c0+c1*t+c2*t**2+(c3+c4*t)*np.cos(2*math.pi*f_year*t+c5)